In [ ]:
import transformers
import huggingface_hub
from huggingface_hub import notebook_login
import ipywidgets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader  
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 

In [3]:
notebook_login()

Loading data and bert uncased model 

In [ ]:
data = pd.read_csv("data/tweet_text_2.csv", encoding = 'cp1252') 
data = data[data.Shortage.notna()]
train_data = data.sample(frac =0.8)
test_data = data[~data.index.isin(train_data.index)]

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Adding a layer to BERT for finetuning for the binary classifcation task

In [8]:
class Model(nn.Module):
        
    def __init__(self, num_labels):
        super().__init__()
        self.bert = model
        self.linear = nn.Linear(768,num_labels)
        self.softmax = nn.Softmax(1)
    def forward(self,token_ids, attention_masks ):
        _,bert_output = model(token_ids, attention_mask=attention_masks,return_dict=False)
        output = self.softmax(self.linear(bert_output))
        return output
   
num_labels = 2    
finetune_model = Model(num_labels)



Will finetune only the last layer in the first model and hence freeze the parameters of other layers

In [9]:
for params in finetune_model.parameters():
    params.requires_grad = False
    
for params in finetune_model.linear.parameters():
    params.requires_grad = True


Setting parameters, optimizer

In [10]:
batch_size = 32
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(finetune_model.parameters(), lr=learning_rate)

Creating a dataset and dataloader

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, token_ids, attention_masks, labels):
        self.tokens = token_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.tokens[idx], self.attention_masks[idx], self.labels[idx]
    
train_sample = tokenizer.batch_encode_plus(list(train_data["TWEET_TEXT"]), padding = "max_length", truncation=True)
test_sample = tokenizer.batch_encode_plus(list(test_data["TWEET_TEXT"]), padding = "max_length", truncation=True)
train_dataset = CustomDataSet(torch.tensor(train_sample["input_ids"]), torch.tensor(train_sample["attention_mask"]), torch.tensor(list(train_data['Shortage'])))
test_dataset = CustomDataSet(torch.tensor(test_sample["input_ids"]), torch.tensor(test_sample["attention_mask"]), torch.tensor(list(test_data['Shortage'])))
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size)


finetuning the model

In [14]:
loss_adam = []
n_epochs = 10

for epoch in range(n_epochs):
    epoch_loss_train, epoch_loss_test, epoch_accuracy_train, epoch_acccuracy_test = 0, 0, 0, 0
    outputs_train, outputs_test, labels_epochs_train, labels_epochs_test  = [], [], [], []
    n = 0 
    for step, batch in enumerate(train_dataloader): 
        n += 1
        batch_loss = 0   
        input_ids, attention_masks, labels = batch
        output = finetune_model(input_ids, attention_masks)
        loss = criterion(output, labels.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"epoch/total epoch : {epoch}/{n_epochs}---- step:{step}----- loss:{loss.item()}")
        epoch_loss_train += loss.item() 
        output = output.detach().numpy()
        labels = labels.detach().numpy()
        outputs_train.append(output)
        labels_epochs_train.append(labels)
    outputs_train = np.concatenate(outputs_train, axis =1)
    label_epochs_train = np.concatenate(label_epochs_train, axis =1)
    epoch_accuracy_train = sum(outputs_train == label_epochs_train)/(outputs_train.shape[0])
    for step, batch in enumerate(test_dataloader):
        input_ids, attention_masks, labels = batch
        output = finetune_model(input_ids, attention_masks)
        loss = criterion(output, labels.long())
        epoch_loss_test += loss.item()
        output = output.detach().numpy()
        labels = labels.detach().numpy()
        outputs_test.append(output)
        labels_epochs_test.append(labels)
    outputs_test = np.concatenate(outputs_test, axis =1)
    label_epochs_test = np.concatenate(label_epochs_test, axis =1)
    epoch_accuracy_test = sum(outputs_test == label_epochs_test)/(outputs_test.shape[0])
    print(f"epoch/total epoch : {epoch}/{n_epochs} --- train loss:{epoch_loss_train/n} --- test loss:{epoch_loss_test/n} --- train accuracy:{epoch_accuracy_train} --- test accuracy:{epoch_accuracy_test}")







KeyboardInterrupt: 

np.float64(0.75)